# Rapport de projet final: Débordement d'égouts
## Équipe 9 - A19
#### _Noboru Yoshida - Mehdi Chaid - Mathieu Giroux-Huppé - Maxime Gosselin_
<br>
20 Décembre 2019

## Table des matières

+ [1. Introduction](#1.)
    + [1.1. Objectifs](#1.1.)  
    + [1.2. Méthode de travail](#1.2.)  
<br>
+ [2. Analyse exploratoire](#2.)
    + [2.1. Motivation](#2.1.)  
    + [2.2. Importation des données](#2.2.)  
    + [2.3. Fonctions utilisées](#2.3.)
    + [2.4. Analyse des données d'ouvrages](#2.4.)  
        + [2.4.1. Données discrètes](#2.4.1.)
        + [2.4.2. Données de Trop-Plein](#2.4.2.)
        + [2.4.3. Données d'Emissaire](#2.4.3.)
    + [2.5. Analyse des données de précipitations](#2.5.)  
        + [2.5.1. Données de précipitations](#2.5.1.)
        + [2.5.2. Somme des précipitations journalières](#2.5.2.)
        + [2.5.3. Taux horaire maximal de précipitations journalières](#2.5.3.)
        + [2.5.4. Taux sur trois heures maximal des précipitations journalières](#2.5.4.)
    + [2.6. Analyse des données de surverses](#2.6.)  
    + [2.7. Isolation des ouvrages d'intérêts](#2.7.)
<br>


+ [3. Traitement des données](#3.)
    + [3.1. Motivation](#3.1.)  
    + [3.2. Fonctions utilisées](#2.2.)  
    + [3.3. Ajout de variables explicatives](#3.3.)
    + [3.4. Standardisation et OneHot](#3.4.)  
    + [3.5. Séparation en ensembles de tests et validation](#3.5.)  
    + [3.6. Isolation des ouvrages d'intérêts](#3.6.)
<br>


+ [4. Sélection de modèles](#4.)
    + [4.1. Motivation](#4.1)
    + [4.2. Choix des modèles](#4.2.)
    + [4.3. Arbres de décision et forêt aléatoire](#4.3.)  
    + [4.4. Régression logistique](#4.4.)  
    + [4.5. Machine à vecteurs de support](#4.5.)  
    + [4.6. Classification bayésienne naive](#4.6.)  
    + [4.7. Ensemble de modèles](#4.7.)  
<br>
+ [5. Retour et conclusion](#5.)
    + [5.1. Retour sur les résultats](#5.1.)  
    + [5.2. Améliorations possibles](#5.2.)
    + [5.3. Difficultés rencontrées](#5.3.)
        + [5.3.1. Interpretation des données](#5.3.1.)
    + [5.4. Retour sur le travail en équipe](#5.4.)  
    + [5.5. Conclusion](#5.5.)  
<br>

+ [6. Références](#refs)

<a id="1."></a>

## 1. Introduction

<a id="1.1."></a>
### 1.1. Objectifs

Ce rapport vise à fournir une prédiction sur les surverses dans plusieurs ouvrages sur l'île de Montréal. Il en existe 170 répartis sur tout le bord de l'île. Dans ce travail, nous nous attarderons sur la prédiction de seulement 5 d'entre-eux, soit les suivants :
- 3260-01D dans Rivière-des-Prairies
- 3350-07D dans Ahunstic
- 4240-01D dans Pointe-aux-Trembles
- 4350-01D dans le Vieux-Montréal
- 4380-01D dans Verdun

Nous avons à notre disposition 3 jeux de données qui nous aiderons à trouver une relation entre la quantité de pluie tombé et les surverses de certain ouvrages. Nous devrons alors entraîner un modèle sur les données de 2013 à 2018, puis prédire sur certaines dates de 2019. Il n'y pas de restrictions au niveau des techniques à utilisés pour le type de modèle à utiliser. Nous avons exploré quelques techniques différentes que nous allons détaillé plus en détail dans la [section 5](#model-selection).

<a id="1.2."></a>
### 1.2. Déroulement de travail

Le projet s'est déroulé de la sorte: 

 1. Tout d'abord, nous avons effectué une analyse superficielle des données afin de comprendre leur sens et de pouvoir commencer la tâche.

 2. Puis, nous avons procédé à une exploration initiale des modèles selon les données que nous avons et en pesant les avantages et inconvénients de chaque modèle afin de déterminer les plus appropriés.

 3. Par la suite, nous avons effectué un traitement initiale des données afin de les intégrer aux modèles.

 4. À cette étape, nous avons appliqué les multiples modèles choisis à nos données traitées afin de les analyser et d'isoler les plus performants.

 5. Nous avons ensuite procédé à une analyse plus poussée afin de trouver des manière d'améliorer nos données.

 6. Nous avons alors traité plus en profondeur les données pour les adapter à la situation et rendre nos modèles plus robustes.

 7. Finalement, nous avons rafiné les modèles selon ces nouvelles données, menant à nos prédictions finales.

Nous aborderons dans ce rapport chacune de ces parties, séparées en 3 groupes: l'analyse exploratoire, le traitement de données et la sélection de modèle. 

<a id="2."></a>
## 2. Analyse exploratoire

<a id="2.1."></a>
### 2.1. Motivation

L'analyse exploratoire est une très importante pour tout exercice en sciences de données. Elle permet de déterminer, entres autres:

- Quelles variables vont jouer un rôle important pour la prédiction.
- À quel point les variables sont corrélées entre elles.
- Si une régularisation est nécessaire pour certaines variables.
- Comment améliorer le temps d'entrainement et les prédictions.

Pour ce faire, nous allons analyser chacune des variables explicatives en utilisant des graphes et des statistiques afin d'appuyer nos résultats et de guider notre prise de décision.

Cette section du rapport, qui est assez longue, peut être ignorée si le lecteur est déjà familier avec les jeux de données utilisés. Elle permet néanmoins de mieux comprendre comment nous avons choisi les variables d'intérêts et comment nous sommes arrivés aux conclusions que nous avons tiré lors de la préparation de modèles.

<a id="2.2."></a>
### 2.2. Importation des données

Installons d'abord les libraires utilisées pour l'analyse exploratoire.

In [ ]:
using Pkg;
Pkg.add("CSV");
Pkg.add("Random");
Pkg.add("DataStructures");
Pkg.add("BenchmarkTools");
Pkg.add("DataFrames");
Pkg.add("Statistics");
Pkg.add("Dates");
Pkg.add("Gadfly");
Pkg.add("IterTools");

Importons maintenant ces librairies dans le rapport.

In [ ]:
using CSV, DataFrames, DataStructures, BenchmarkTools, Statistics, Dates, Gadfly, Random, IterTools;

Configuration de l'outil de visualisation.

In [ ]:
set_default_plot_size(25cm, 13cm)

<a id="2.3."></a>
### 2.3. Fonctions utilisées

Nous définissons ici les fonctions utilisées pour l'analyse exploratoire de données. Ces fonctions sont implémentées dans le fichier suivant afin de ne pas polluer le rapport.

In [27]:
include("utils/precipitation.jl");

<a id="2.3.1."></a>
#### 2.3.1. random_first

Cette function est un remplacement à la fonction first qui nous permet d'obtenir un échantillon aléatoire de données.

In [ ]:
function random_first(df::DataFrame, nb::Int64, fts::Array=[])
    if length(fts) == 0
        first(df[shuffle(1:size(df, 1)),:], nb)
    else
        first(df[shuffle(1:size(df, 1)),fts], nb)
    end
end

<a id="2.3.2."></a>
#### 2.3.2. maximum_3

#### TODO: Describe

<a id="2.4."></a>
### 2.4. Analyse des données d'ouvrages

Commençons par importer le fichier de données d'ouvrages et renommons les colonnes pour qu'elles soient plus facile à traiter. Le fichier concerné est _ouvrages-surverses.csv_. Pour plus d'informations sur ce jeu de données: http://donnees.ville.montreal.qc.ca/dataset/ouvrage-surverse.

In [ ]:
ouvrages = CSV.read("data/ouvrages-surverses.csv");
colnames = ["N_ENV", "ID_SOMA", "ID_OUVRAGE", "NOM", "SOMA_SEC", "REGION", "TP_X", "TP_Y", "TP_Z", "TP_LAT", "TP_LNG", "EMI_X", "EMI_Y", "EMI_LNG", "EMI_LAT"];
names!(ouvrages, Symbol.(colnames));

<a id="2.4.1."></a>
#### 2.4.1. Données discrètes

Tout d'abord, procédons à une analyse des données discrètes des ouvrages. Les données discrètes de ce jeu de données sont:

- N_ENV: Identifiant de l'ouvrage Environnement Canada.
- ID_SOMA: Identifiant de l'ouvrage selon SOMAEU (Gouvernement du Québec).
- ID_OUVRAGE: Identifiant de l'ouvrage selon la ville.
- NOM: Nom de l'ouvrage.
- SOMA_SEC: Nom du secteur SOMAEU où se trouve l'ouvrage.
- REGION: Nom de la région (arrondissement ou municipalité) où se trouve l'ouvrage.

Analysons brièvement chacune de ces variables afin de déterminer si elles pourraient jouer le rôle de variable explicative dans notre situation.

*N_ENV, ID_SOMA, ID_OUVRAGE, NOM:*

In [ ]:
random_first(ouvrages, 10, [:N_ENV, :ID_SOMA, :ID_OUVRAGE, :NOM])

Comme on peut le voir, chaque ouvrage possède une valeur unique pour ces colonnes. Celà fait du sens, compte tenu qu'il s'agit d'un identifiant unique à l'ouvrage, exprimé sous plusieurs normes.
<br>

En raison du faible pouvoir prédictif d'une telle propriété, dûe à l'unicité de chaque valeur, et de la haute corrélation entre les variables, nous avons décidé de ne pas inclure ces variables dans notre modèle. Nous conservons évidemment l'identifiant de l'ouvrage (_ID_OUVRAGE_) qui va permettre d'identifier chaque ouvrage et de joindre plusieurs jeux de données ensemble, mais les autres ne semblent pas posséder de pouvoir prédictif significatif.
<br>

Il est à noter que le numéro contenu dans _N_ENV_ et _ID_SOMA_, ainsi que les 3 lettres dans _ID_SOMA_ pourraient tous deux permettrent de grouper certains ouvrages ensembles ou de les localiser dans une carte, mais d'autres variables explicatives abordées à la section [2.4.1. Données discrètes](#2.4.1.) et [2.4.2. Données de Trop-Plein](#2.4.2.) s'occupent déjà de cette localisation. 

_SOMA_SEC, REGION_

Observons maintenant les effets de ces deux variables discrètes. Tout d'abord, le secteur selon la norme SOMAEU. Visualisons la carte de Montréal selon les positions de Trop-Pleins, identifiant le secteur SOMAEU par couleur.

In [ ]:
plot(ouvrages,x=:TP_LNG, y=:TP_LAT, Geom.point, color= :SOMA_SEC)

Comme on peut le voir, la majorité des ouvrages sont groupés sous le secteur _OMAEU de Montreal (Station Jean-R.-Marcotte_). Ce problème d'imbalancement de classes pourrait s'avérer problématique. Effectuons maintenant la même tâche pour la séparation de secteurs selon leur arrondissement ou municipalité.

In [ ]:
plot(ouvrages,x=:TP_LNG, y=:TP_LAT, Geom.point, color= :REGION)

Les secteurs sont beaucoups mieux balancés. Cette variable explicative permet de grouper ensemble un certain nombres d'ouvrages, ce qui pourrait s'avérer utile pour la prédiction.

On remarque cependant que certaines régions ne contiennent que très peu d'ouvrages. Il pourrait être intéressant de grouper ensemble des régions contigues afin d'obtenir un nombre d'ouvrage par région plus balancé encore.

<a id="2.4.2."></a>
#### 2.4.2. Données de trop-plein

Passons maintenant à l'analyse des données de trop-plein. Nous avons ici 5 variables explicatives pour le trop-plein.

- TP_X: Coordonnée en X selon la norme NAD83 MTM8.
- TP_Y: Coordonnée en Y selon la norme NAD83 MTM8
- TP_Z: Coordonnée en Z selon la norme NAD83 MTM8.
- TP_LNG: Longitude selon la norme WGS84.
- TP_LAT: Latitude selon la norme WGS84.

Pour ce qui est de l'interprétation des trops-pleins, bien que nous ayons eu certaines difficultés à correctement interpréter le sens de ce terme en comparaison avec les émissaires, comme discuté dans la section [5.3. Difficultés rencontrées](#5.3.), des recherches plus poussées en ligne nous ont guidé vers le sens du trop-plein dans ce contexte: Il s'agit ici de l'ouvrage qui permet aux eaux non dirigées vers la station d'épuration d'être évacuées vers le milieu récepteur naturel<sup>[[1]](#refs)</sup>. 

Les couples de coordonnées [X, Y] et [LNG, LAT] semblent redondantes, testons cette hypothèse.

*TP_X, TP_Y, TP_LNG, TP_LAT:*

In [ ]:
plt_x_y = plot(ouvrages, x=:TP_X, y=:TP_Y, Geom.point);
plt_lng_lat = plot(ouvrages, x=:TP_LNG, y=:TP_LAT, Geom.point);
hstack(plt_x_y, plt_lng_lat)

Comme on peut le voir sur les graphiques, ces données sont en effet plutôt similaires et mappent toutes les deux le contour de l'Ile de Montréal, où sont situés les Trop-Pleins. Nous avons choisi de ne garder pour l'instant que les latitudes et longitudes, ce format étant consistent avec les données présentes dans un autre jeu de données.

*TP_Z:*

Observons maintenant la répartition des hauteurs de trop-plein.

In [ ]:
plot(ouvrages, x=:TP_Z, Geom.histogram(bincount=50), Guide.xlabel("Hauteur de trop-plein"),Guide.ylabel("Fréquence"))

Une analyse sur les données de hauteur nous informent que cette information est manquante pour une partie des ouvrages. 

In [ ]:
describe(ouvrages[!, :TP_Z])

Comme on peut le voir, environ 10% des données sont manquantes. Une solution à ce problème est proposée dans la section [3. Traitement des données](#3.).

<a id="2.4.3."></a>
#### 2.4.3. Données d'emissaire

Concluons l'analyse des données d'ouvrages avec les variables concernant l'émissaire. Selon la description des données accompagnant le jeu de données sur le site de Montréal, le positionnement géographique des ouvrages de débordement est associé aux émissaires dans cet ensemble de données<sup>[[2]](#refs)</sup>. 

Encore une fois, ces coordonnées sont présentes selon les deux normes, avec le couple [X,Y] ou le couple [LNG, LAT]. Nous choisissons le second afin de rester consistent avec nos autres choix jusqu'à présent.

Ces données contiennent des valeurs manquantes, comme l'indique la description des colonnes.

In [ ]:
describe(ouvrages[!, :EMI_LNG])

In [ ]:
describe(ouvrages[!, :EMI_LAT])

Cela correspond aux indications présentes sur le site web. Le traitement des données manquantes sera abordé dans la section [3. Traitement des données](#3.). Pour l'instant, nous allons ignorer les données manquantes afin d'observer le graphique des émissaires.

In [ ]:
emi_lng_no_missing = collect(skipmissing(ouvrages[!, :EMI_LNG]));
emi_lat_no_missing = collect(skipmissing(ouvrages[!, :EMI_LAT]));
plot(x=emi_lng_no_missing, y=emi_lat_no_missing, Geom.point)

Comme on peut le voir, la forme ressemble fortement à celle obtenue avec les données de trop-pleins. Stackons les deux pour voir la différence et comparons verticalement le résultat lorsque chacun est par dessus l'autre.

In [ ]:
tp_first_emi = plot(layer(ouvrages, x=:TP_LNG, y=:TP_LAT, Geom.point, order=1, Theme(default_color=colorant="orange")),
     layer(x=emi_lng_no_missing, y=emi_lat_no_missing, Geom.point, order=2, Theme(default_color=colorant="blue")),
     Guide.title("Émissaire par dessus"), Guide.xlabel(nothing),Guide.ylabel(nothing));

emi_first_tp = plot(layer(ouvrages, x=:TP_LNG, y=:TP_LAT, Geom.point, order=2, Theme(default_color=colorant="orange")),
     layer(x=emi_lng_no_missing, y=emi_lat_no_missing, Geom.point, order=1, Theme(default_color=colorant="blue")),
     Guide.title("Trop-plein par dessus"), Guide.xlabel(nothing),Guide.ylabel(nothing));

hstack(tp_first_emi, emi_first_tp)

Comme on peut le voir, ces coordonnées sont assez semblables. Utiliser les deux dans notre modèle pourrait donc entrainer des problèmes de corrélation. Nous avons, pour la majorité du temps de travail, utilisé les coordonnées de trop-pleins afin d'effectuer nos prédictions, n'utilisant donc pas les données d'émissaires dans notre modèle. Ce choix provient du fait que les données d'émissaires sont parfois manquantes (10% du temps).
<br>

Une autre approche utilisée récemment nous permet de ne pas avoir à choisir entre les deux et de ne pas utiliser de données géographiques. Cette approche sera discutée dans la section [2.7. Isolation des ouvrages d'intérêts](#2.7).

<a id="2.5."></a>
### 2.5. Analyse des données de précipitations

        + [2.5.1. Précipitations par station](#2.5.1.)
        + [2.5.2. Somme des précipitations journalières](#2.5.2.)
        + [2.5.3. Taux horaire maximum de précipitations journalières](#2.5.3.)
        + [2.5.4. Taux horaire sur trois heures continues maximum](#2.5.4.)

<a id="2.5.1."></a>
#### 2.5.1. Données de précipitation

Passons à l'analyse des données de précipitations à certaines stations pluviométrique. Nous devons effectuer des fonctions d'aggrégation sur ce tableau avant de pouvoir l'intégré dans notre jeu de donnée de nos modèles. En effet, ce tableau contient les données de précipitation par heure pour chacune des cinqs stations météorologiques près de montréal. Cependant, notre jeu de données des features est seulement par jour. Nous allons donc applique 3 fonctions d'aggrégation différente, soit : Somme, Maximum et Maximum3h. 

Ce jeu de données a été préparé par notre professeur à partir des données météos d'environnement Canada<sup>[[3]](#refs)</sup> et contient la date sous le format yyyy-mm-jj, l'heure, ainsi que la quantité de pluie au dixième de millimètre pour chacune des cinqs stations pluviométriques suivante :
- McTavish (7024745)
- Ste-Anne-de-Bellevue (702FHL8)
- Montreal/Pierre Elliott Trudeau Intl (702S006)
- Montreal/St-Hubert (7027329)
- L’Assomption (7014160)

Les dates contenues dans ce dataset comprennent les années 2013 à 2019. Par contre, nous devons entraîner notre modèle sur les années avant 2019, et prédire sur toute l'année 2019 (de mai à octobre). Il est donc important de noter que cette table contient des données de l'ensemble d'entraînement et de test.

Commençons par importer les données de précipitations et renommer la colonne St-Hubert afin qu'elle soit plus facile à traiter.

In [ ]:
precipitations = CSV.read("data/precipitations.csv",missingstring="-99999");
rename!(precipitations, Symbol("St-Hubert")=>:StHubert);

Nous ne sommes intéressés que par les mois de Mai à Octobre, inclusivement. Analysons les valeurs associés à ce lapse de temps.

In [ ]:
precipitations = filter(row -> month(row.date) > 4, precipitations);
precipitations = filter(row -> month(row.date) < 11, precipitations);
random_first(precipitations, 5)

Comme on peut l'observer ci-dessous, plusieurs de ces données sont manquantes.

In [ ]:
describe(precipitations)

In [ ]:
describe(precipitations[!, :McTavish])

 La stratégie de résolution des données manquantes est discutée dans la section [3. Traitement des données](#3.). Pour l'instant, gardons seulement les données complêtes à des fins d'observations.

In [ ]:
dropmissing!(precipitations);
size(precipitations, 1)

<a id="2.5.2."></a>
#### 2.5.2. Somme des précipitations journalières

Commençons par obtenir la somme des précipitations dans une journée, par station pluviométrique.

In [ ]:
pcp_sum = by(precipitations, :date,  
            McTavish = :McTavish=>sum, 
            Bellevue = :Bellevue=>sum,
            Assomption = :Assomption=>sum, 
            Trudeau = :Trudeau=>sum, 
            StHubert = :StHubert=>sum);

Il y a beaucoup d'éléments sur ce graphique. Une particularité de Gadfly est qu'il nous permet de choisir les séries à afficher. On peut cliquer donc cliquer sur la station d'intérêt afin de filtrer les résultats. On peut remarquer sur ce graphique que les stations ont des tendances similaires, mais pas totalement identiques. Si nous prennons McTavish et Assomption pour l'année 2016, nous pouvons voir que la majorité des pics se superposent, avec quelques exceptions où des pics plus bas apparaissent seulement pour une station.

In [ ]:
df_for_plot = filter(row -> year(row.date) == 2016, pcp_sum);
df_for_plot = melt(df_for_plot, :date)
plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable, Guide.title("Somme des précipitations des stations météo pour 2016"), Guide.xlabel("Date (j)"), Guide.ylabel("Précipitation (mm 10^-1)"))

Il semblerait donc, jusqu'à présent, que McTavish reçoive plus de précipitations en moyenne que les autres stations. Voyons si cette tendance se maintient pour les autres années.

In [ ]:
df_for_plot = melt(pcp_sum, :date)
plot(df_for_plot, x=:date, y=:value, Geom.point, color=:variable, Guide.title("Somme des précipitations des stations météo de 2013 à 2018"), Guide.xlabel("Date (j)"),  Guide.ylabel("Précipitation (mm 10^-1)"))

La tendance semble se conserver, mais l'année 2019 est très différente, beaucoup d'informations sont manquantes. Analysons séparément l'année 2019.

In [ ]:
df_for_plot = filter(row -> year(row.date) == 2019, pcp_sum);
df_for_plot = melt(df_for_plot, :date)
plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable, Guide.title("Somme des précipitations des stations météo pour 2019"), Guide.xlabel("Date (j)"), Guide.ylabel("Précipitation (mm 10^-1)"))

Comme on peut le voir, nous ne possédons des données de précipitations que pour la majorité du mois de Mai. En réalité, une analyse de la table de précipitation nous informe que nous possédons les informations couvrant la majorité de l'année pour certaines stations, mais d'autres, comme Bellevue, ne sont couvertes que jusqu'en Mai. Une solution à ce problème sera abordé à la section [3. Traitement des données](#3.).

<a id="2.5.3."></a>
#### 2.5.3. Taux horaire maximal des précipitations journalières

Passons au taux horaire maximal par jour des précipitations.

In [21]:
pcp_max = by(precipitations, :date,  
            McTavish = :McTavish=>maximum,
            Bellevue = :Bellevue=>maximum, 
            Assomption = :Assomption=>maximum,
            Trudeau = :Trudeau=>maximum,
            StHubert = :StHubert=>maximum);

Visualisation de ces informations pour l'année 2016:

In [ ]:
df_for_plot = filter(row -> year(row.date) == 2016, pcp_max);
df_for_plot = melt(df_for_plot, :date)
plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable, Guide.title("Taux horaire maximal des précipitations pour 2016"), Guide.xlabel("Date (j)"), Guide.ylabel("Précipitation (mm 10^-1)"))

Le graphique semble suggérer une tendance similaire à celle que la somme des précipitations journalières nous procure. Le reste des informations étant assez similaires entre la somme et le taux horaire maximal, nous ne montrerons pas de graphes additionels dans cette sous-section.

<a id="2.5.4."></a>
#### 2.5.4. Taux sur trois heures maximal des précipitations journalières

Finalement, voyons le taux sur trois heures maximal des précipitations.

In [32]:
pcp_max3 = by(precipitations, :date,
                McTavish = :McTavish=>maximum3,
                Bellevue = :Bellevue=>maximum3,
                Assomption = :Assomption=>maximum3,
                Trudeau = :Trudeau=>maximum3,
                StHubert = :StHubert=>maximum3);

Visualisation de ces informations pour l'année 2016:

In [ ]:
df_for_plot = filter(row -> year(row.date) == 2016, pcp_max3);
df_for_plot = melt(df_for_plot, :date)
plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable, Guide.title("Taux maximal sur 3 heures pour 2016"), Guide.xlabel("Date (j)"), Guide.ylabel("Précipitation (mm 10^-1)"))

Encore une fois, nous pouvons voir des positions de pics similaires entre les trois graphiques, avec des hauteurs de pics différentes compte tenu des types de valeurs variables.

En effet, la somme des précipitations pour une journée sera toujours supérieure ou égale au taux maximal sur 3h qui lui même sera toujours supérieur ou égal au taux horaire maximal. Démontrons cette hypothèse avec les données pour McTavish en 2018:

In [ ]:
date_to_pt = 2018;
loc = :McTavish
mct_sum = pcp_sum[:,[1,2]]
rename!(mct_sum, loc => :Sum);
mct_sum = filter(row -> year(row.date) == date_to_pt, mct_sum);

mct_max = pcp_max[:,[1,2]]
rename!(mct_max,loc => :Max);
mct_max = filter(row -> year(row.date) == date_to_pt, mct_max);

mct_max3 = pcp_max3[:,[1,2]]
rename!(mct_max3,loc => :Max3);
mct_max3 = filter(row -> year(row.date) == date_to_pt, mct_max3);

df_for_plot = join(mct_sum, mct_max3, on = :date);
df_for_plot = join(df_for_plot, mct_max, on = :date);
df_for_plot = melt(df_for_plot, :date)


plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable)

<a id="2.6."></a>
### 2.6. Analyse des données de surverses [TODO: Mettre à jour table des matières]

<a id="2.6.1"></a>
#### 2.6.1. Chargement des surverses pour analyse

Enfin, nous chargeons les données du fichier <i>Surverses.csv</i>. Ce jeu de donnée provient des données ouvertes de Montréal : http://donnees.ville.montreal.qc.ca/dataset/debordement. Le fichier ne correspond pas tout à fait aux données en ligne. Il y a eu des transformations (jointures) apportées par notre professeur. Le fichier est structuré de la sorte:
<br>

**NO_Ouvrage** : (String) L'idendifiant unique de chacun des ouvrages <br>
**Date** : (Date) La date sous le format yyyy-mm-jj de l'observation <br>
**Surverse** : (Bool) Si l'ouvrage a surversé à cette date <br>
**Raison** : (String) Abréviation de la raison de la surverse. Les options sont variables, Ex:
- "U" : Urgence;
- "Inconnue" : La raison est inconnue;
- "TS" : Déversement par temps sec;
- "P" : Débordement du à la pluie; 
- "N" : Rejet à la rivière des Prairies;
- "S" : Rejet au fleuve St-Laurent;

Au final, nous ne garderons que les trois premières colonnes. La colonne NO_Ouvrage sera renommé afin d'être consistente avec l'identifiant de l'ouvrage des autres jeux de données. De plus, nous ne garderons que les surverses dûes à la pluie, à un débordement par temps sec ou à une raison inconnue que nous considérerons comme étant dûe à la pluie.

Chargeons les données de surverses:

In [44]:
surverses = CSV.read("data/surverses.csv", missingstring="-99999");

Nous ne gardons que les mois de Mai à Octobre inclusivement.

In [45]:
surverses = filter(row -> month(row.DATE) > 4, surverses);
surverses = filter(row -> month(row.DATE) < 11, surverses);

Conservons les surverses dûes à la pluie, au temps sec ou de raison inconnue.

In [46]:
raison = coalesce.(surverses[:,:RAISON],"Inconnue");
surverses[!,:RAISON] = raison;

surverses = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], surverses);
select!(surverses, [:NO_OUVRAGE, :DATE, :SURVERSE]);

Les données de surverses manquantes sont retirées: On ne peut rien analyser avec celles ci.

In [47]:
surverses = dropmissing(surverses, disallowmissing=true);
rename!(surverses, :NO_OUVRAGE => :ID_OUVRAGE);

<a id="2.6.2."></a>
#### 2.6.2. Lien entre ouvrages et surverses

Existe t'il des ouvrages qui sont plus susceptibles à des surverses que d'autres? C'est ce que nous allons chercher à déterminer dans cette sous-section. Commençons par joindre ces deux jeux de données ensemble.

In [77]:
ouvrages_surverses = join(ouvrages, surverses, on =:ID_OUVRAGE);
select!(ouvrages_surverses, [:ID_OUVRAGE, :SURVERSE, :TP_LNG, :TP_LAT, :TP_Z]);

Voyons si la position des ouvrages influence le nombre de surverses reçues.

In [ ]:
n_surverses_by_ouvrage = by(ouvrages_surverses, :ID_OUVRAGE, 
    SURVERSE = :SURVERSE => sum,
    TP_LNG = :TP_LNG => mean, 
    TP_LAT = :TP_LAT => mean,
    TP_Z = :TP_Z => mean);
plot(n_surverses_by_ouvrage, x=:TP_LNG, y=:TP_LAT, Geom.point, color=:SURVERSE)

À première vue, il seble que la moitié droite de l'Île de Montréal soit plus touchée par des surverses que la moitié gauche. On observe aussi le plus de surverses au croisement milieu de l'Île. La position semble donc influencer la quantité de surverses qu'un ouvrage subit durant l'année. On se rend compte en revanche que c'est principalement la longitude qui joue en rôle dans la quantité de surverses plutôt que la longitude.

Ceci n'est évidemment qu'une observation qui semble indiquer une corrélation, mais ne constitue pas une preuve de corrélation. Nous tiendrons néanmoins compte de ce résultat dans notre traitement de données puis dans notre sélection de modèles.

Quand est il de la hauteur du trop plein? Observons son influence sur la quantité de surverses.

In [ ]:
dropmissing!(n_surverses_by_ouvrage, disallowmissing=true);
plot(n_surverses_by_ouvrage, x=:TP_Z, y=:SURVERSE, Geom.hair)

Contrairement à notre intuition, la hauteur ne semble pas jouer un rôle si important sur la quantité de surverses d'un trop-plein.

<a id="2.6.3."></a>
#### 2.6.3. Lien entre précipitations et surverses

Trouvons maintenant le lien entre le taux de surverses et la quantité de précipitation reçus par les ouvrages. Le facteur commun direct entre les deux est la date, groupons donc ces données par dates.

On commence par renommer la colonne :date des précipitations afin de pouvoir effectuer la jointure.

In [ ]:
rename!(pcp_sum, :date => :DATE)
rename!(pcp_max, :date => :DATE)
rename!(pcp_max3, :date => :DATE)

On effectue la jointure et on groupe par date afin de visualiser les données.

In [125]:
pcp_sum_surverses = join(pcp_sum, surverses, on=:DATE)
select!(pcp_sum_surverses, Not([:ID_OUVRAGE]))
n_surverses_by_date = by(pcp_sum_surverses, :DATE, 
                            McTavish = :McTavish => first,
                            Bellevue = :Bellevue => first,
                            Assomption = :Assomption => first,
                            Trudeau = :Trudeau => first,
                            StHubert = :StHubert => first,
                            SURVERSE = :SURVERSE => sum);

Observons l'effet de la quantité de pluie reçus dans une journée à McTavish sur le taux de précipitation.

In [ ]:
plot(n_surverses_by_date, x=:McTavish, y=:SURVERSE, Geom.point)

Comme nous pouvons le remarquer, il existe une relation bien définie entre la quantité de pluie reçue et le nombre de surverses dans une journée.

Voyons si ce phénomène est présent pour les autres stations pluviométriques.

In [ ]:
bellevue_plot = plot(n_surverses_by_date, x=:Bellevue, y=:SURVERSE, Geom.point);
assomption_plot = plot(n_surverses_by_date, x=:Assomption, y=:SURVERSE, Geom.point);
trudeau_plot = plot(n_surverses_by_date, x=:Trudeau, y=:SURVERSE, Geom.point);
st_hubert_plot = plot(n_surverses_by_date, x=:StHubert, y=:SURVERSE, Geom.point);
gridstack([bellevue_plot assomption_plot; trudeau_plot st_hubert_plot])

Bien que la relation ne soit pas aussi clairement définie partout, il existe toutefois un lien entre la quantité reçue par une station et le nombre de surverses sur l'Île de Montréal. Nous tiendrons compte de cette information lors du traitement des données et de l'entrainement des modèles.

Ce même évènement devrait se produire pour pcp_max et pcp_max3, étant donné qu'ils sont très corrélés avec pcp_sum. Assurons nous que ce soit le cas avec pcp_max.

In [136]:
pcp_max_surverses = join(pcp_max, surverses, on=:DATE)
select!(pcp_max_surverses, Not([:ID_OUVRAGE]))
n_surverses_by_date = by(pcp_max_surverses, :DATE, 
                            McTavish = :McTavish => first,
                            Bellevue = :Bellevue => first,
                            Assomption = :Assomption => first,
                            Trudeau = :Trudeau => first,
                            StHubert = :StHubert => first,
                            SURVERSE = :SURVERSE => sum);

Reprenons la station McTavish pour analyser le phénomène.

In [ ]:
plot(n_surverses_by_date, x=:McTavish, y=:SURVERSE, Geom.point)

Comme on pouvait l'imaginer, le graphe est très similaire à ce que l'on pouvait imaginer. 
Afin de garder le rapport concis, nous ne démontrerons pas ce résultat pour les autres stations et pour pcp_max3 ici.

<a id="2.7."></a>
### 2.7. Isolation des ouvrages d'intérêts

#### TODO

<a id="3."></a>
## 3. Traitement des données

#### TODO

<a id="4."></a>
## 4. Sélection de modèles

<a id="4.1."></a>
### 4.1. Motivation

Maintenant que les données sont prêtes à être utilisées, il est temps de passer à l'étape de sélection de modèles. En effet, le choix du ou des modèles est crucial dans le domaine de la prédiction car, selon les données reçues, certains modèles vont mener à des meilleures prédictions. Il est donc primordial de bien déterminer le type de modèle qui serait adapté à notre situation afin d'obtenir les meilleures performances et résultats possibles.

<a id="4.2."></a>
### 4.2. Choix des modèles

Les modèles que nous avons considéré pour cet exercice sont:

- Les arbres de décision
- La forêt aléatoire
- La machine à vecteur de support (SVM)
- La régression logistique
- La classification bayésienne naive
- L'ensemble de modèles

Chacun de ces modèles sera présenté dans une sous-section de la section [4. Sélection de modèles](#4.), où les avantages et désavantages du modèle seront présentés. Pour les modèles aillant été retenus, nous allons détailler les notions théoriques sur lesquels reposent ces modèles et comment nous les avons appliqué. Pour les modèles non retenus, nous expliqueront pourquoi ils n'ont pas été retenu en faveur des autres.

<a id="4.3."></a>
### 4.3. Arbres de décision et forêt aléatoire [TODO: METTRE A JOUR TABLE DES MATIERES]

<a id="4.3.1."></a>
#### 4.3.1. Théorie des arbres de décision

Le modèle d'arbres de décision est un algorithme d'apprentissage machine très puissant capable d'effectuer des tâches de classification, mais aussi de régression. Dans notre cas, c'est la partie classification qui nous intéresse ici car nous cherchons à déterminer si une surverse a eu lieu ou pas, selon certaines variables explicatives.

Les avantages des arbres de décision sont:
- A
- B
- C

Les désavantages des arbres de décision sont:
- A
- B
- C

Les arbres de décision agissent sur chaque variable en **TODO: CONTINUE**

<a id="4.3.2."></a>
#### 4.3.2. Théorie des forêts aléatoires

<a id="4.3.3."></a>
#### 4.3.3. Définition des fonctions utilitaires

<a id="4.3.4."></a>
#### 4.3.4. Application à l'ensemble de données

<a id="4.3.5."></a>
#### 4.3.5. Application aux ouvrages isolés

<a id="4.4."></a>
### 4.4. Régression logistique

<a id="4.5."></a>
### 4.5. Machine à vecteurs de support

<a id="4.6."></a>
### 4.6. Classification bayésienne naive

<a id="4.7."></a>
### 4.7. Ensemble de modèles

<a id="5."></a>
## 5. Retour et conclusion

<a id="5.3."></a>
### 5.3. Difficultés rencontrées

Lors de cet exercice, nous avons rencontré un certain nombre de difficultés qui nous en ralentit dans la réalisation du projet. Nous allons discuter ici de ces difficultés.

<a id="5.3.1."></a>
#### 5.3.1. Interprétation des données

Une des plus grande difficultés de ce projet fut de correctement interpréter les données. En effet, une erreur de notre part fut de confondre la signification d'émissaire et de trop-plein. Ne sachant pas initialement ce que ces termes représentaient, une recherche Google Image nous a mené à penser que le trop-plein était une sorte de conteneur dans lequel l'eau s'accumulait et l'émissaire comme étant le tuyeau de déversement vers le fleuve. 
<br>

Cela nous a ainsi mené à effectuer des suppositions qui s'avéraient erronées et ont faussé nos résultats pendant une bonne partie de l'exercice.

<a id="refs"></a>
## 6. Références

## TODO: Référence style APA ou IEEE
+ [1]: http://collections.banq.qc.ca/ark:/52327/bs44911

+ [2]: http://donnees.ville.montreal.qc.ca/dataset/ouvrage-surverse

+ [3]: https://climat.meteo.gc.ca/climate_data/hourly_data_f.html?hlyRange=2008-01-08%7C2019-11-12&dlyRange=2002-12-23%7C2019-11-12&mlyRange=%7C&StationID=30165&Prov=QC&urlExtension=_f.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=17&searchMethod=contains&Month=11&Day=12&txtStationName=montreal&timeframe=1&Year=2019
